In [ ]:
using CairoMakie
using LilGuys
using Arya

In [ ]:
using DataFrames, CSV
using OrderedCollections

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis/"

In [ ]:
import TOML

In [ ]:
"""
    load_orbit(filename)

"""
function load_orbit(filename)
    out = Output(joinpath(modelsdir, filename))

    df = DataFrame(
        time = out.times * T2GYR, 
        x = out.x_cen[1, :],
        y = out.x_cen[2, :],
        z = out.x_cen[3, :],
        v_x = out.v_cen[1, :] * V2KMS,
        v_y = out.v_cen[2, :] * V2KMS,
        v_z = out.v_cen[3, :] * V2KMS,
        )

    df[!, :R] = @. sqrt(df.x^2 + df.y^2)
    df[!, :r] = @.sqrt(df.x^2 + df.y^2 + df.z^2)

    L = LilGuys.calc_L_spec(out.x_cen, out.v_cen)

    df[!, :Lx] = L[1, :]
    df[!, :Ly] = L[2, :]
    df[!, :Lz] = L[3, :]

    pot = agama.Potential(joinpath(modelsdir, filename, "simulation/agama_potential.ini"))

    act, ang = get_actions(pot, out.x_cen, out.v_cen)

    df[!, :Jr] = act[1, :]
    df[!, :Jz] = act[2, :]
    df[!, :Jϕ] = act[3, :]
    
    df[!, :Θr] = ang[1, :]
    df[!, :Θz] = ang[2, :]
    df[!, :Θϕ] = ang[3, :]

    idx_f = TOML.parsefile(joinpath(modelsdir, filename, "orbital_properties.toml"))["idx_f"]


    snap_cen = Snapshot(out.x_cen, out.v_cen, zeros(size(out.x_cen, 2)))
    obs_c = LilGuys.to_gaia(snap_cen, add_centre=false)

    df[!, :ra] = obs_c.ra
    df[!, :dec] = obs_c.dec
    df[!, :distance] = obs_c.distance
    df[!, :pmra] = obs_c.pmra
    df[!, :pmdec] = obs_c.pmdec
    df[!, :radial_velocity] = obs_c.radial_velocity

    return df[1:idx_f, :]
end

In [ ]:
"""
    load_orbit_expected(filename)

"""
function load_orbit_expected(filename)
    path = joinpath(modelsdir, "../simulations/", filename)
    df = CSV.read(path, DataFrame)

    df[!, :R] = @. sqrt(df.x^2 + df.y^2)
    df[!, :r] = @.sqrt(df.x^2 + df.y^2 + df.z^2)
    df[!, :time] = df.t * T2GYR

    return df
end

In [ ]:
function compare_orbits_2d(orbits; xname = "x", yname = "y", xunits = "kpc", yunits="kpc",
        aspect=DataAspect(), kwargs...
    )
    fig = Figure()
    
    ax = Axis(fig[1, 1];
        xlabel = "$xname / $xunits",
        ylabel = "$yname / $yunits", 
        aspect=aspect,
        kwargs...
    )
    
    for (label, orbit) in orbits
        x = orbit[:, Symbol(xname)]
        y = orbit[:, Symbol(yname)]
        lines!(x, y, label = label)

    end

    Legend(fig[1, 2], ax)

    fig
end

In [ ]:
orbits = [
    "mean" => load_orbit_expected("sculptor/1e7_V31_r3.2/orbit_mean/orbit.csv"),
    "smallperi" => load_orbit_expected("sculptor/1e7_V31_r3.2/orbit_smallperi/orbit.csv"),
    ];

In [ ]:
compare_orbits_2d(orbits, yname="z", xname="y", limits=(-100,100,-100,100))

In [ ]:
compare_orbits_2d(orbits, yname="r", xname="time", aspect=nothing, xunits="Gyr", limits=(0, 10, 35, 110))

In [ ]:
orbits = [
    "iorio fiducial" => load_orbit_expected(ENV["DWARFS_ROOT"] * "/analysis/sculptor/mc_orbits/iorio/orbit1.csv"),
    "iorio extreme" => load_orbit_expected(ENV["DWARFS_ROOT"] * "/analysis/sculptor/mc_orbits/iorio/orbit2.csv"),
    ];

In [ ]:
compare_orbits_2d(orbits, yname="r", xname="time", aspect=nothing, xunits="Gyr", limits=(0, 10, 35, 110))

In [ ]:
compare_orbits_2d(orbits, yname="z", xname="y", limits=(-100,100,-100,100))

In [ ]:
orbits = [
    "mean" => load_orbit("sculptor/1e7_V31_r3.2/orbit_mean"),
    "smallperi" => load_orbit("sculptor/1e7_V31_r3.2/orbit_smallperi"),
    #"heavy" => load_orbit("sculptor/1e6_V40_r5.9/orbit_mean"),
    ];

In [ ]:
compare_orbits_2d(orbits, yname="r", xname="time", aspect=nothing, xunits="Gyr", limits=(0, 10, 35, 110))

In [ ]:
compare_orbits_2d(orbits, yname="z", xname="y", limits=(-100,100,-100,100))

In [ ]:
compare_orbits_2d(orbits, yname="z", xname="R")

In [ ]:
compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="r", aspect=nothing)

# Action comparisons

In [ ]:
include(ENV["DWARFS_ROOT"] * "/utils/agama_utils.jl")

In [ ]:
function compare_orbits_all(orbits)
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="r", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="R", xunits="kpc", yname="z") |> display
    
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Lx", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Ly", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Lz", aspect=nothing) |> display
    
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Jr", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Jz", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Jϕ", aspect=nothing) |> display

    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Θr", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Θz", aspect=nothing) |> display
    compare_orbits_2d(orbits, xname="time", xunits="Gyr", yname="Θϕ", aspect=nothing) |> display

end
    

In [ ]:
orbits = OrderedDict(
    "mean" => load_orbit("ursa_minor/1e5_v37_r5.0/orbit_mean.2"),
    "mean2" => load_orbit("ursa_minor/1e6_v37_r5.0/orbit_mean.2"),
    #"heavy" => load_orbit("sculptor/1e6_V40_r5.9/orbit_mean"),
    )

In [ ]:
compare_orbits_all(orbits)

In [ ]:
orbits = OrderedDict(
    "point" =>
    "mean0" => load_orbit("ursa_minor/1e6_v37_r5.0/orbit_mean"),
    "mean1" => load_orbit("ursa_minor/1e5_v37_r5.0/orbit_mean.1"),
    "mean2" => load_orbit("ursa_minor/1e5_v37_r5.0/orbit_mean.2"),
    "mean2_hr" => load_orbit("ursa_minor/1e6_v37_r5.0/orbit_mean.2"),
    )

In [ ]:
compare_orbits_all(orbits)

In [ ]:
obs_props_file = joinpath(ENV["DWARFS_ROOT"], "observations/ursa_minor/observed_properties.toml")

In [ ]:
obs_props = TOML.parsefile(obs_props_file)

let
    icrs = LilGuys.coord_from_file(obs_props_file)
    gc = LilGuys.transform(LilGuys.Galactocentric, icrs)
    
    obs_props["R"] = @. sqrt(gc.x^2 + gc.y^2)
    obs_props["r"] = @.sqrt(gc.x^2 + gc.y^2 + gc.z^2)

    obs_props["z"] = gc.z
    
    x_cen = LilGuys.position_of(gc)
    v_cen = LilGuys.velocity_of(gc) / V2KMS
    L = LilGuys.calc_L_spec(x_cen, v_cen)
    
    obs_props["Lx"] = L[1]
    obs_props["Ly"] = L[2]
    obs_props["Lz"] = L[3]
    
    pot = agama.Potential(joinpath(ENV["DWARFS_ROOT"], "agama/potentials/EP2020.ini"))

    act, ang = get_actions(pot, x_cen, v_cen)
    
    obs_props["Jr"] = act[1, 1]
    obs_props["Jz"] = act[2, 1]
    obs_props["Jϕ"] = act[3, 1]
    
    obs_props["Θr"] = ang[1, 1]
    obs_props["Θz"] = ang[2, 1]
    obs_props["Θϕ"] = ang[3, 1]
end

obs_props

In [ ]:
function plot_gradients(orbits;
        ys=["ra", "dec", "distance", "pmra", "pmdec", "radial_velocity"]
    )
    
    xs = ["Jr", "Jz", "Jϕ", "Θr", "Θz", "Θϕ"]

    fig = Figure(size=(1000, 1000))

    for j in eachindex(xs)
        x = xs[j]
        for i in eachindex(ys)
    
            y = ys[i]
            ax = Axis(fig[i,j], 
                xlabel = x,
                ylabel = y
            )

            hlines!(obs_props[y])
            if y*"_err" ∈ keys(obs_props)
                hspan!(obs_props[y] - obs_props[y*"_err"], obs_props[y] + obs_props[y*"_err"], color=(COLORS[1], 0.2))
            end

            
            for (k, orbit) in orbits
                scatter!(orbit[1, x], orbit[end, y])
                text!(orbit[1, x], orbit[end, y], text=k)
            end

    
            if i < lastindex(ys)
                hidexdecorations!(ax, grid=false)
            end
            if j != 1
                hideydecorations!(ax, grid=false)
            end
        end
    end
    
    fig
end

In [ ]:
plot_gradients(orbits)

In [ ]:
plot_gradients(orbits, ys=["Jr", "Jz", "Jϕ", "Θr", "Θz", "Θϕ" ])

In [ ]:
plot_gradients(orbits, ys=["R", "z", "Lx", "Ly", "Lz"])

In [ ]:
orbits = OrderedDict(
    "point" => load_orbit_expected("ursa_minor/1e6_v37_r5.0/orbit_mean/orbit.csv"),
     "nbody" => load_orbit("ursa_minor/1e6_v37_r5.0/orbit_mean.2")
    )

for (k) in keys(orbits)
    orbits[k].time .-= orbits[k].time[end]
end

In [ ]:
compare_orbits_2d(orbits, xname="y", yname="z")

In [ ]:
compare_orbits_2d(orbits, xname="time", yname="r", aspect=nothing)